In [2]:
import riak

client = riak.RiakClient()

/usr/local/lib/python2.7/dist-packages/riak-2.2.0-py2.7.egg/riak/security.py:35: UserWarning: Found OpenSSL 1.0.1f 6 Jan 2014 version, but expected at least OpenSSL 1.0.1g.  Security may not support TLS 1.2.


In [2]:
client.ping()

True

In [4]:
bucket = client.bucket('rooms')

In [5]:
STYLES = ["single", "double", "queen", "king","suite"]

In [6]:
from random import randint, seed

seed(10)

for floor in range(1,100):
    current_rooms_block = floor * 100
    #print "Making rooms #{} - #{}".format(current_rooms_block, current_rooms_block + 100)
    
    # Put 100 rooms on each floor (huge hotel!)
    for room in range(1,100):
        # Create a unique room number as the key
        key = str(current_rooms_block + room)
        ro = riak.RiakObject(client, bucket, key)
        
        style = STYLES[randint(0, len(STYLES)-1)]
        capacity = randint(0, 8) + 1
              
        ro.content_type = "application/json"
        ro.data = {'style': style, 'capacity': capacity}
        ro.store()
    

In [25]:
mapReduce = riak.mapreduce.RiakMapReduce(client)
print mapReduce

In [26]:
mapReduce.add("rooms", {"101", "102", "103"})

In [95]:
mapReduce.map("""
    function(v) { 
        /* From the Riak object, pull data and parse it as JSON */ 
        var parsed_data = JSON.parse(v.values[0].data); 
        var data = {};  
        /* Key capacity number by room style string */ 
        data[parsed_data.style] = parsed_data.capacity; 
        return [data]; 
    }""")

In [96]:
mapReduce.run()

[{u'1': 14, u'2': 1}]

In [97]:
mapReduce = riak.mapreduce.RiakMapReduce(client)

mapReduce.add("rooms", {"101", "102", "103", "201"}) 

mapReduce.map("""
    function(v) {
        var parsed_data = JSON.parse(v.values[0].data); 
        var data = {};
        var floor = ~~(parseInt(v.key) / 100);
        data[floor] = parsed_data.capacity; 
        return [data];
      }
    """)

mapReduce.reduce("""
    function(v) {
            var totals = {};
            for (var i in v) {
              for(var floor in v[i]) {
                if( totals[floor] ) 
                  totals[floor] += v[i][floor];
                else
                  totals[floor] = v[i][floor];
              } 
            }

            return [totals];
          }
    """)

mapReduce.run()

[{u'1': 14, u'2': 1}]

In [98]:
bucket = client.bucket('my_functions')

obj = riak.RiakObject(client, bucket, 'map_capacity')
obj.content_type = 'application/json'
obj.encoded_data = """function(v) {
    var parsed_data = JSON.parse(v.values[0].data);
    var data = {};
    data[parsed_data.style] = parsed_data.capacity;
    return [data];
}"""

obj.store()

In [99]:
!curl  http://localhost:8098/riak/my_functions/map_capacity

function(v) {
    var parsed_data = JSON.parse(v.values[0].data);
    var data = {};
    data[parsed_data.style] = parsed_data.capacity;
    return [data];
}

In [117]:
mapReduce = riak.mapreduce.RiakMapReduce(client)

mapReduce.add("rooms", {"101", "102", "103", "201"}) 
mapReduce.map( 
"my_functions.map_capacity"
)

mapReduce.run()

RiakError: '{"phase":0,"error":"[{<<\\"lineno\\">>,1},{<<\\"message\\">>,<<\\"ReferenceError: my_functions is not defined\\">>},{<<\\"source\\">>,<<\\"unknown\\">>}]","input":"{ok,{r_object,<<\\"rooms\\">>,<<\\"201\\">>,[{r_content,{dict,4,16,16,8,80,48,{[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]},{{[],[],[],[],[],[],[],[],[],[],[[<<\\"content-type\\">>,97,112,112,108,105,99,97,116,105,111,110,47,106,115,111,110],[<<\\"X-Riak-VTag\\">>,54,70,65,106,65,71,53,115,115,83,69,110,88,106,69,82,49,67,56,98,82,98]],[[<<\\"index\\">>]],[],[[<<\\"X-Riak-Last-Modified\\">>|{1443,460673,438063}]],[],[]}}},<<\\"{\\\\\\"style\\\\\\": \\\\\\"suite\\\\\\", \\\\\\"capacity\\\\\\": 1}\\">>}],[{<<35,9,254,249,13,134,190,165,0,6,182,...>>,...}],...},...}"}'

In [118]:
%%file ~/notebooks/riak/input.json
{
"inputs":[
["rooms","101"],["rooms","102"],["rooms","103"]
],
"query":[
{"map":{
"language":"javascript",
"bucket":"my_functions",
"key":"map_capacity"
}}
]
}

Overwriting /home/learner/notebooks/riak/input.json


In [119]:
!curl -X POST -H "content-type:application/json" \
http://localhost:8098/mapred --data "@/home/learner/notebooks/riak/input.json"

[{"queen":4},{"queen":2},{"suite":8}]

In [86]:
#"key_filters":[["string_to_int"], ["less_than", 1000]]

In [87]:
%%file ~/notebooks/riak/input.json
{
"inputs":{
    "bucket":"rooms",
    "key_filters": [["string_to_int"], ["between", 4200, 4399]]
},
"query":[
{"map":{
"language":"javascript",
"bucket":"my_functions",
"key":"map_capacity"
}},
{"reduce":{
"language":"javascript",
"source":
"function(v) {
var totals = {};
for (var i in v) {
for(var style in v[i]) {
if( totals[style] ) totals[style] += v[i][style];
else totals[style] = v[i][style];
}
}
return [totals];
}"
}}
]
}

Overwriting /home/learner/notebooks/riak/input.json


In [88]:
!curl -X POST -H "content-type:application/json" \
http://localhost:8098/mapred --data "@/home/learner/notebooks/riak/input.json"

[{"queen":201,"suite":209,"double":169,"single":228,"king":146}]

In [22]:
mapReduce = riak.mapreduce.RiakMapReduce(client)

mapReduce.add("rooms", {"101", "102", "103", "201"}) 
#mapReduce.map_values()
#mapReduce.map("Riak.mapValues")
mapReduce.map_values_json()

'''
mapReduce.map("""
    function(v) {
        var parsed_data = JSON.parse(v.values[0].data); 
        var data = {};
        var floor = ~~(parseInt(v.key) / 100);
        data[floor] = parsed_data.capacity; 
        return [data];
        }
    """)
'''

#mapReduce.reduce("Riak.reduceSum")

mapReduce.run()

[{u'capacity': 1, u'style': u'suite'},
 {u'capacity': 8, u'style': u'suite'},
 {u'capacity': 4, u'style': u'queen'},
 {u'capacity': 2, u'style': u'queen'}]

In [39]:
mapReduce = riak.mapreduce.RiakMapReduce(client)

mapReduce.add("rooms", {"101", "102", "103", "201"}) 
mapReduce.map("Riak.mapByFields", {"arg": [ "capacity" ]})



#mapReduce.reduce("Riak.reduceSum")

mapReduce.run()